# Azure AI Agent with Existing Thread Example

This notebook demonstrates working with pre-existing conversation threads by providing thread IDs for thread reuse patterns.

## Features Covered:
- Creating and managing persistent conversation threads
- Working with existing thread IDs
- Thread lifecycle management (create, use, delete)
- Conversation continuity across sessions
- Thread initialization and validation

## Prerequisites

Before running this notebook, ensure you have:

1. **Azure AI Project**: Access to an Azure AI Foundry project with deployed models
2. **Authentication**: Azure CLI installed and authenticated (`az login --use-device-code`)
3. **Environment Variables**: Set up your `.env` file with connection details

If you need to use a different tenant, specify the tenant ID:
```bash
az login --tenant <tenant-id>
```

This example demonstrates how to work with existing threads to maintain conversation continuity.

## Import Required Libraries

First, let's import all the necessary libraries and modules:

In [ ]:
#!about
#r "nuget: Azure.Identity, 1.18.0-beta.2"
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.*"
#r "nuget: Microsoft.Agents.AI.AzureAI, 1.0.0-preview.*"
#r "nuget: dotenv.net"

using Azure.Identity;
using Microsoft.Agents.AI;
using Azure.AI.Projects;
using Azure.AI.Projects.OpenAI;
using Microsoft.Extensions.AI;
using dotenv.net;
using System.IO;
using System.Threading.Tasks;
using System.ComponentModel;
using System.Text.Json;
using OpenAI.Assistants;
using OpenAI.Responses;
using OpenAI;
using System.ClientModel;


## Initial Setup
We'll start by loading environment variables and initializing an **AIProjectClient** so we can do all the agent-related actions. Let's do it! 🎉

In [ ]:
DotEnv.Load(new DotEnvOptions(envFilePaths: new[] { Path.Combine(".", "..", "..","..", ".env") })); 
var tenantId = Environment.GetEnvironmentVariable("TENANT_ID");
var projectEndpoint = Environment.GetEnvironmentVariable("AI_FOUNDRY_PROJECT_ENDPOINT");
var modelDeploymentName = Environment.GetEnvironmentVariable("MODEL_DEPLOYMENT_NAME");
// Create the Credentials
var credentialOptions = new InteractiveBrowserCredentialOptions
{
    TenantId = tenantId
};
var credential = new InteractiveBrowserCredential(credentialOptions);

AIProjectClient aiProjectClient = new AIProjectClient(new Uri(projectEndpoint),credential);

AgentThread? agentThread = null;
string AgentName = "WeatherAgent";

## Define Function Tools and AI Agent

Define our tools and agent for use

In [ ]:
[Description("Get the current weather for a given location.")]
string GetWeather([Description("The location to get the weather for.")]string location)
{
    // Simulate getting weather data
    var conditions = new[] { "sunny", "cloudy", "rainy", "windy", "stormy" };
    var random = new Random();
    var condition = conditions[random.Next(conditions.Length)];
    
    return $"🌤️ The weather in {location} is {condition} with a high of {random.Next(60, 101)}°F.";
}

var tool = AIFunctionFactory.Create(GetWeather);

var agent = await aiProjectClient.CreateAIAgentAsync(
    name: AgentName,
    instructions: "You are a helpful weather agent.",
    model: modelDeploymentName!,
    tools: [tool]
);

## Create and Use Existing Thread

This example shows how to:
1. Create a thread that persists beyond the current session
2. Use the thread ID to continue conversations
3. Properly clean up thread resources

In [ ]:
async Task CreateOrReuseExistingThread(ChatClientAgent agent, string threadId)
{
    Console.WriteLine("=== Azure AI Chat Client with Exsting Thread ===");
    if (!string.IsNullOrEmpty(threadId))
    {
        Console.WriteLine($"Reusing existing thread with ID: {threadId}");
        agentThread = agent.GetNewThread(threadId);
        return;
    }
    agentThread = agent.GetNewThread();
}

## Execute the agent and store the thread Id for later

In this example, we'll create a conversation for reuse, and pass it to the agent for execution

In [ ]:
var openAIClient = aiProjectClient.GetProjectOpenAIClient();

var conversationClient = openAIClient.GetProjectConversationsClient();

var conversation = await conversationClient.CreateProjectConversationAsync();

var conversationId = conversation.Value.Id;
Console.WriteLine($"Created new conversation with ID: {conversationId}");
await CreateOrReuseExistingThread(agent, conversationId);

Console.WriteLine($"👤 User: What's the weather like in Seattle?");
var response = await agent.RunAsync("What's the weather like in Seattle?", agentThread!);
Console.WriteLine($"🤖 Agent: {response.Text}");

## Reuse the conversation and send more requests

This shows how to reuse the previous thread Id we stored and continue our conversation with the agent.

In [ ]:
await CreateOrReuseExistingThread(agent, conversationId);
Console.WriteLine($"👤 User: How about in New York?");
var response = await agent.RunAsync("How about in New York?", agentThread!);
Console.WriteLine($"🤖 Agent: {response.Text}");

Console.WriteLine("=== End of Azure AI Chat Client with Existing Thread ===");

## Viewing full thread

Since we created the thread using the portal sdk, we can now visit the portal and view the thread and the full conversation.

1. Launch the [Foundry Portal](https://ai.azure.com)
2. Go to the Build Tab
3. Find the "WeatherAgent" defined in the portal
4. Go to the "Traces" tab, and look for the conversation id in the previous cell trace

You should be able to see the full conversation with all messages sent and received by the Agent now.



## Key Takeaways

1. **Persistent Threads**: Threads can persist beyond a single session, enabling conversation continuity
2. **Thread Initialization**: Always verify that threads are properly initialized before use
3. **Conversation Context**: Threads maintain conversation history, allowing for contextual responses
4. **Resource Management**: Always clean up threads when they're no longer needed
5. **Multi-Threading**: You can manage multiple separate conversations using different thread IDs
6. **Thread Reuse**: Threads can be reused across different agent instances

## Best Practices

1. **Thread Lifecycle**: Create threads when starting conversations and delete them when finished
2. **Error Handling**: Use try-finally blocks to ensure thread cleanup
3. **Thread Validation**: Always check if threads are initialized before using them
4. **Context Awareness**: Leverage thread history for more contextual conversations
5. **Resource Monitoring**: Monitor thread usage to avoid unnecessary resource consumption

## Use Cases

- **Customer Support**: Maintaining conversation history across support sessions
- **Multi-Session Chats**: Continuing conversations across different user sessions
- **Conversation Branching**: Managing multiple conversation topics simultaneously
- **Context Preservation**: Keeping conversation context for better user experience
- **Session Management**: Managing different conversation sessions for different users